 # ACE Next — Interactive Demo



 This notebook walks through the refactored `ace_next` pipeline.

 It covers:



 1. **Runners** — `ACE` (full pipeline) and `TraceAnalyser` (learning-only)

 2. **Steps** — individual pipeline steps and `learning_tail()`

 3. **Manual pipeline construction** — composing steps by hand

 4. **Custom environments** — writing your own evaluator

 5. **Checkpointing & deduplication** — production features

 6. **Skillbook persistence** — save / reload

 7. **TraceAnalyser** — learning from pre-recorded traces



 **Requirements:** `uv sync` from the repo root.

 Set your LLM API key before running:

 ```bash

 export OPENAI_API_KEY="sk-..."

 ```

 ## 1. Setup & Imports

In [1]:
import os
import sys
import tempfile
from pathlib import Path

import nest_asyncio

nest_asyncio.apply()

# Ensure the project root is on sys.path so `ace`, `ace_next`, and `pipeline`
# are importable regardless of where the notebook kernel starts.
_here = Path(__file__).resolve().parent if "__file__" in dir() else Path.cwd()
_root = _here
for _p in [_here] + list(_here.parents):
    if (_p / "pipeline" / "__init__.py").exists():
        _root = _p
        break
sys.path.insert(0, str(_root))

from dotenv import load_dotenv

load_dotenv(_root / ".env")

print(f"Project root: {_root}")
print("Setup OK")


Project root: /home/david/Desktop/projects/Kayba/agentic-context-engine
Setup OK


 ## 2. Core Imports



 Everything lives in `ace_next` — fully self-contained, zero cross-imports.

In [2]:
from ace_next import (
    # Runners
    ACE,
    TraceAnalyser,
    # Role implementations
    Agent,
    Reflector,
    SkillManager,
    # LLM providers
    LiteLLMClient,
    # Core types
    Sample,
    Skillbook,
    SimpleEnvironment,
    TaskEnvironment,
    EnvironmentResult,
    AgentOutput,
    # Context types
    ACEStepContext,
    SkillbookView,
)

print("All imports OK")


/home/david/Desktop/projects/Kayba/agentic-context-engine/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports OK


 ## 3. Configure the LLM Client



 We use LiteLLM which supports 100+ providers. Swap the model string

 for any provider: `gpt-4o-mini`, `claude-sonnet-4-5-20250929`,

 `bedrock/us.anthropic.claude-haiku-4-5-20251001-v1:0`, etc.

In [3]:
MODEL = os.getenv("ACE_MODEL", "us.anthropic.claude-haiku-4-5-20251001-v1:0")
client = LiteLLMClient(model=MODEL)

print(f"LLM client ready: {MODEL}")


LLM client ready: us.anthropic.claude-haiku-4-5-20251001-v1:0


 ## 4. Build Roles



 The three ACE roles share the same LLM client. Each is independently

 customisable (prompt templates, retries, etc.).

In [4]:
agent = Agent(client)
reflector = Reflector(client)
skill_manager = SkillManager(client)

print("Roles created: Agent, Reflector, SkillManager")


Roles created: Agent, Reflector, SkillManager


 ## 5. Define Training Samples

In [5]:
samples = [
    Sample(question="What is the capital of France?", ground_truth="Paris"),
    Sample(question="What is the capital of Japan?", ground_truth="Tokyo"),
    Sample(question="What is the capital of Brazil?", ground_truth="Brasilia"),
    Sample(question="What is the capital of Australia?", ground_truth="Canberra"),
    Sample(question="What is the capital of Nigeria?", ground_truth="Abuja"),
]

print(f"Prepared {len(samples)} training samples")


Prepared 5 training samples


 ---

 ## 6. ACE Runner — Full Adaptive Pipeline



 The `ACE` runner is the full closed-loop pipeline:

 ```

 Agent → Evaluate → Reflect → Tag → Update → Apply

 ```



 It takes `Sample` objects and an optional `TaskEnvironment`.

 ### 6a. With SimpleEnvironment



 `SimpleEnvironment` checks if the ground truth appears in the agent's

 answer (case-insensitive substring match).

In [ ]:
skillbook = Skillbook()

ace = ACE.from_roles(
    agent=agent,
    reflector=reflector,
    skill_manager=skill_manager,
    environment=SimpleEnvironment(),
    skillbook=skillbook,
)

results = ace.run(samples[:3], epochs=1)

print(f"Processed {len(results)} samples\n")
for r in results:
    if r.error:
        print(f"  ERROR at {r.failed_at}: {r.error}")
    elif r.output:
        ctx: ACEStepContext = r.output
        answer = ctx.agent_output.final_answer if ctx.agent_output else "N/A"
        print(f"  Q: {r.sample.question}")
        print(f"  A: {answer}")


13:55:00 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


13:55:06 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/api/v1/session/redirect/projects/?trace_id=0699d9fa-af5c-7693-8000-2f4cdb4d27df&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL2FwaS8=.
13:55:06 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


...

13:55:35 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


OPIK: Failed to process CreateSpansBatchMessage. Error: headers: {'date': 'Tue, 24 Feb 2026 12:55:36 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=lz8YF7VVKVIDN0GWRnfpcJ5hgNF3qKNtOmU2wWBRx0R1Jg/L1Z8lpz2B6Q6v78bjRNw5E/QXP0hs8ogWgx9yYzAUuWjO39ca5pB9pxvN7WZPf2wR7J+dvAgC+7Qa; Expires=Tue, 03 Mar 2026 12:55:36 GMT; Path=/, AWSALBCORS=lz8YF7VVKVIDN0GWRnfpcJ5hgNF3qKNtOmU2wWBRx0R1Jg/L1Z8lpz2B6Q6v78bjRNw5E/QXP0hs8ogWgx9yYzAUuWjO39ca5pB9pxvN7WZPf2wR7J+dvAgC+7Qa; Expires=Tue, 03 Mar 2026 12:55:36 GMT; Path=/; SameSite=None; Secure', 'server': 'nginx', 'comet-ver': 'deprecated_1771937736887', 'comet-app-server': 'cometml-react-f54d994cd-dbvgn', 'access-control-expose-headers': 'Comet-Ver, Comet-App-Server', 'content-security-policy': "base-uri 'self'; connect-src 'self' ws: wss: https: https://*.comet.com https://rs.fullstory.com https://*.reo.dev https://www.google-analytics.com; default-src 'self'; font-src 'self' data: https:

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
Processed 3 samples

  Q: What is the capital of France?
  A: The capital of France is Paris.
  Q: What is the capital of Japan?
  A: Tokyo is the capital of Japan.
  Q: What is the capital of Brazil?
  A: The capital of Brazil is Brasília.


OPIK: Failed to process CreateSpansBatchMessage. Error: headers: {'date': 'Tue, 24 Feb 2026 12:55:44 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=3of7cAVPXDO8ilejqa7se2TmqMlIGBxiogiv+tiExuSesoThT20JiCAPkBo6C2k2OmJJGggB6taxNQkgazx1wcO/afcfSQYtAejbTdx9bbzLPUu2LnMKfbeTU5Iw; Expires=Tue, 03 Mar 2026 12:55:44 GMT; Path=/, AWSALBCORS=3of7cAVPXDO8ilejqa7se2TmqMlIGBxiogiv+tiExuSesoThT20JiCAPkBo6C2k2OmJJGggB6taxNQkgazx1wcO/afcfSQYtAejbTdx9bbzLPUu2LnMKfbeTU5Iw; Expires=Tue, 03 Mar 2026 12:55:44 GMT; Path=/; SameSite=None; Secure', 'server': 'nginx', 'comet-ver': 'deprecated_1771937744900', 'comet-app-server': 'cometml-react-f54d994cd-dbvgn', 'access-control-expose-headers': 'Comet-Ver, Comet-App-Server', 'content-security-policy': "base-uri 'self'; connect-src 'self' ws: wss: https: https://*.comet.com https://rs.fullstory.com https://*.reo.dev https://www.google-analytics.com; default-src 'self'; font-src 'self' data: https:

In [7]:
print(f"\nSkillbook after 1 epoch:")
print(f"  Stats: {skillbook.stats()}")
for skill in skillbook.skills()[:5]:
    print(f"  - [{skill.id}] {skill.content}")



Skillbook after 1 epoch:
  Stats: {'sections': 2, 'skills': 5, 'tags': {'helpful': 5, 'harmful': 0, 'neutral': 0}}
  - [factual_queries-00001] Classify problem type before selecting reasoning approach
  - [factual_queries-00002] Use direct knowledge retrieval for straightforward factual questions
  - [factual_queries-00003] Verify facts against authoritative references before formulation
  - [factual_recall-00004] Answer factual questions directly with specific supporting details
  - [factual_queries-00005] Match output format exactly: remove accents per specification


 ### 6b. Custom Environment



 Create your own evaluator by subclassing `TaskEnvironment`.

In [8]:
class ExactMatchEnvironment(TaskEnvironment):
    """Strict evaluation: answer must exactly match ground truth."""

    def evaluate(self, sample: Sample, agent_output: AgentOutput) -> EnvironmentResult:
        expected = (sample.ground_truth or "").strip().lower()
        predicted = agent_output.final_answer.strip().lower()
        correct = expected in predicted

        return EnvironmentResult(
            feedback="Correct!" if correct else f"Wrong. Expected: {sample.ground_truth}",
            ground_truth=sample.ground_truth,
            metrics={"accuracy": 1.0 if correct else 0.0},
        )


print("ExactMatchEnvironment defined")


ExactMatchEnvironment defined


In [ ]:
skillbook2 = Skillbook()

ace2 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=ExactMatchEnvironment(),
    skillbook=skillbook2,
)

results2 = ace2.run(samples[:2], epochs=1)

for r in results2:
    if r.output:
        ctx = r.output
        print(f"  Q: {r.sample.question}")
        print(f"  A: {ctx.agent_output.final_answer if ctx.agent_output else 'N/A'}")
        if ctx.reflection:
            print(f"  Insight: {ctx.reflection.key_insight}")
        print()


13:56:02 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


13:56:09 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


13:56:09 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


...

13:56:29 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


OPIK: Failed to process CreateTraceBatchMessage. Error: headers: {'date': 'Tue, 24 Feb 2026 12:56:30 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=dyNvovtF3LXqG3IHQenFlAC7Sm5/P08m6K19fR7arWREiUpRXkVMoGMn448hip228tzNhgZPD5vU2AKmPH33O0sQOT7LMCRHurhzTtR7SHGZWqKci7LUnoGPbZDu; Expires=Tue, 03 Mar 2026 12:56:30 GMT; Path=/, AWSALBCORS=dyNvovtF3LXqG3IHQenFlAC7Sm5/P08m6K19fR7arWREiUpRXkVMoGMn448hip228tzNhgZPD5vU2AKmPH33O0sQOT7LMCRHurhzTtR7SHGZWqKci7LUnoGPbZDu; Expires=Tue, 03 Mar 2026 12:56:30 GMT; Path=/; SameSite=None; Secure', 'server': 'nginx', 'comet-ver': 'deprecated_1771937790978', 'comet-app-server': 'cometml-react-f54d994cd-4v5bz', 'access-control-expose-headers': 'Comet-Ver, Comet-App-Server', 'content-security-policy': "base-uri 'self'; connect-src 'self' ws: wss: https: https://*.comet.com https://rs.fullstory.com https://*.reo.dev https://www.google-analytics.com; default-src 'self'; font-src 'self' data: https:

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
  Q: What is the capital of France?
  A: The capital of France is Paris.
  Insight: Matching reasoning complexity to problem type is more efficient than applying uniform strategic frameworks—factual queries benefit from direct retrieval and verification rather than decomposition strategies.

  Q: What is the capital of Japan?
  A: Tokyo is the capital of Japan.
  Insight: Agent demonstrates appropriate problem classification capability—recognizing when factual knowledge retrieval is sufficient and strategic problem-solving is unnecessary. This classification accuracy prevents unnecessary complexity and enables efficient response generation.



OPIK: Failed to process CreateSpansBatchMessage. Error: headers: {'date': 'Tue, 24 Feb 2026 12:56:40 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=EUlsOCLXt5E943i1yGhaIOIkcF2ODuNWhR70w5xPY5qHLBYeaUqtehOkMEyeq37uHsAsz4BWXxsil0fQ7Oiqz2Q5ae2kzfMULp3A0nuIDHlBkOBQACMQFyTGOfRE; Expires=Tue, 03 Mar 2026 12:56:40 GMT; Path=/, AWSALBCORS=EUlsOCLXt5E943i1yGhaIOIkcF2ODuNWhR70w5xPY5qHLBYeaUqtehOkMEyeq37uHsAsz4BWXxsil0fQ7Oiqz2Q5ae2kzfMULp3A0nuIDHlBkOBQACMQFyTGOfRE; Expires=Tue, 03 Mar 2026 12:56:40 GMT; Path=/; SameSite=None; Secure', 'server': 'nginx', 'comet-ver': 'deprecated_1771937800995', 'comet-app-server': 'cometml-react-f54d994cd-4v5bz', 'access-control-expose-headers': 'Comet-Ver, Comet-App-Server', 'content-security-policy': "base-uri 'self'; connect-src 'self' ws: wss: https: https://*.comet.com https://rs.fullstory.com https://*.reo.dev https://www.google-analytics.com; default-src 'self'; font-src 'self' data: https:

 ### 6c. Without Environment



 When no environment is provided, `EvaluateStep` is a no-op. The Reflector

 still learns from ground-truth comparison in the trace.

In [ ]:
skillbook3 = Skillbook()

ace3 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    skillbook=skillbook3,
    # No environment — EvaluateStep passes through
)

results3 = ace3.run(samples[:2], epochs=1)
print(f"Processed {len(results3)} samples (no environment)")
print(f"Skills learned: {skillbook3.stats()}")


13:58:07 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


13:58:14 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


13:58:14 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


...

13:58:32 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


OPIK: Failed to process CreateSpansBatchMessage. Error: headers: {'date': 'Tue, 24 Feb 2026 12:58:33 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=6mua1PEKx6VOBWtaTouUYoLM2OnXcrwqCYjoCRGiGrP4kfzjDhp2O4QzhvVAaAHWTO9YPu3h+aOpIISl9P0uCdib8ltO0Wqb9uY/m3mF68n6CKVwb6r7ckeShcGV; Expires=Tue, 03 Mar 2026 12:58:33 GMT; Path=/, AWSALBCORS=6mua1PEKx6VOBWtaTouUYoLM2OnXcrwqCYjoCRGiGrP4kfzjDhp2O4QzhvVAaAHWTO9YPu3h+aOpIISl9P0uCdib8ltO0Wqb9uY/m3mF68n6CKVwb6r7ckeShcGV; Expires=Tue, 03 Mar 2026 12:58:33 GMT; Path=/; SameSite=None; Secure', 'server': 'nginx', 'comet-ver': 'deprecated_1771937913179', 'comet-app-server': 'cometml-react-f54d994cd-dbvgn', 'access-control-expose-headers': 'Comet-Ver, Comet-App-Server', 'content-security-policy': "base-uri 'self'; connect-src 'self' ws: wss: https: https://*.comet.com https://rs.fullstory.com https://*.reo.dev https://www.google-analytics.com; default-src 'self'; font-src 'self' data: https:

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
Processed 2 samples (no environment)
Skills learned: {'sections': 3, 'skills': 6, 'tags': {'helpful': 6, 'harmful': 0, 'neutral': 0}}


OPIK: Failed to process CreateSpansBatchMessage. Error: headers: {'date': 'Tue, 24 Feb 2026 12:58:43 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive', 'set-cookie': 'AWSALB=4krATYN+knZ2u0EENSw1cv1pF+UxbM2GquWh0jVIt5+zX2NoGaUYQ0hSHlcmZMfmJ5SNJo2U5f38XrWHH81tf6s3sWvSCuVWd9CzELdKx+VevMRb4nGYn866dKH8; Expires=Tue, 03 Mar 2026 12:58:43 GMT; Path=/, AWSALBCORS=4krATYN+knZ2u0EENSw1cv1pF+UxbM2GquWh0jVIt5+zX2NoGaUYQ0hSHlcmZMfmJ5SNJo2U5f38XrWHH81tf6s3sWvSCuVWd9CzELdKx+VevMRb4nGYn866dKH8; Expires=Tue, 03 Mar 2026 12:58:43 GMT; Path=/; SameSite=None; Secure', 'server': 'nginx', 'comet-ver': 'deprecated_1771937923190', 'comet-app-server': 'cometml-react-f54d994cd-4v5bz', 'access-control-expose-headers': 'Comet-Ver, Comet-App-Server', 'content-security-policy': "base-uri 'self'; connect-src 'self' ws: wss: https: https://*.comet.com https://rs.fullstory.com https://*.reo.dev https://www.google-analytics.com; default-src 'self'; font-src 'self' data: https:

 ### 6d. Multi-Epoch Training



 Multiple epochs let the agent revisit samples with an evolving skillbook.

 Skills accumulate and refine across passes.

In [ ]:
skillbook4 = Skillbook()

ace4 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=SimpleEnvironment(),
    skillbook=skillbook4,
)

results4 = ace4.run(samples, epochs=2)

print(f"Total results across 2 epochs: {len(results4)}")
print(f"Skills learned: {skillbook4.stats()}")

# Print per-epoch accuracy
for epoch in range(1, 3):
    epoch_results = [r for r in results4 if r.output and r.output.epoch == epoch]
    correct = sum(
        1 for r in epoch_results
        if r.output and r.output.agent_output
        and (r.sample.ground_truth or "").lower() in r.output.agent_output.final_answer.lower()
    )
    print(f"  Epoch {epoch}: {correct}/{len(epoch_results)} correct")


 ---

 ## 7. Manual Step-by-Step Pipeline



 Under the hood, runners compose `Pipeline` objects from individual steps.

 Here we build one by hand to see exactly what each step does.

In [ ]:
from pipeline import Pipeline
from ace_next.steps import (
    AgentStep,
    EvaluateStep,
    ReflectStep,
    TagStep,
    UpdateStep,
    ApplyStep,
    learning_tail,
)

skillbook5 = Skillbook()
env = SimpleEnvironment()

# Build the full pipeline manually
pipe = Pipeline(
    [
        AgentStep(Agent(client)),
        EvaluateStep(env),
        *learning_tail(Reflector(client), SkillManager(client), skillbook5),
    ]
)

print(f"Pipeline steps: {len(pipe._steps)}")
print(f"  requires: {pipe.requires}")
print(f"  provides: {pipe.provides}")


 ### Run a single sample through the manual pipeline

In [ ]:
sample = samples[0]

# Build the context the same way ACE._build_context() does
ctx = ACEStepContext(
    sample=sample,
    skillbook=SkillbookView(skillbook5),
    epoch=1,
    total_epochs=1,
    step_index=0,
    total_steps=1,
    global_sample_index=0,
)

print(f"Before pipeline:")
print(f"  Skills: {skillbook5.stats()}")
print(f"  agent_output: {ctx.agent_output}")

# Run the full pipeline on a single context
from pipeline.protocol import SampleResult

results_manual = pipe.run([ctx])

print(f"\nAfter pipeline:")
for r in results_manual:
    if r.error:
        print(f"  ERROR: {r.error}")
    elif r.output:
        out: ACEStepContext = r.output
        print(f"  Agent answer:      {out.agent_output.final_answer if out.agent_output else 'N/A'}")
        print(f"  Reflector insight:  {out.reflection.key_insight if out.reflection else 'N/A'}")
        print(f"  Skills now:        {skillbook5.stats()}")


 ### Using `learning_tail()` as a building block



 `learning_tail()` returns the standard learning steps:

 `[ReflectStep, TagStep, UpdateStep, ApplyStep]` with optional

 deduplication and checkpoint steps appended.

In [ ]:
skillbook6 = Skillbook()

tail = learning_tail(
    Reflector(client),
    SkillManager(client),
    skillbook6,
)

print(f"learning_tail() returns {len(tail)} steps:")
for step in tail:
    print(f"  - {type(step).__name__}")


 ---

 ## 8. Checkpointing



 Save the skillbook every N successful samples so you can resume after

 interruption or compare skillbook evolution over time.

In [ ]:
skillbook7 = Skillbook()

with tempfile.TemporaryDirectory() as tmpdir:
    ace7 = ACE.from_roles(
        agent=Agent(client),
        reflector=Reflector(client),
        skill_manager=SkillManager(client),
        environment=SimpleEnvironment(),
        skillbook=skillbook7,
        checkpoint_dir=tmpdir,
        checkpoint_interval=2,  # save every 2 successful samples
    )

    results7 = ace7.run(samples, epochs=1)

    saved = sorted(Path(tmpdir).glob("*.json"))
    print("Checkpoint files:")
    for f in saved:
        print(f"  {f.name}  ({f.stat().st_size} bytes)")


 ---

 ## 9. Deduplication



 Merge near-duplicate skills to keep the skillbook compact. The

 `DeduplicationManager` runs periodically during training.

In [ ]:
from ace_next import DeduplicationManager, SimilarityDetector
from ace_next.protocols import DeduplicationConfig

skillbook8 = Skillbook()

dedup = DeduplicationManager(
    DeduplicationConfig(similarity_threshold=0.85)
)

ace8 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=SimpleEnvironment(),
    skillbook=skillbook8,
    dedup_manager=dedup,
    dedup_interval=3,  # run dedup every 3 samples
)

results8 = ace8.run(samples, epochs=1)

print(f"Skills after training with dedup: {skillbook8.stats()}")


 ---

 ## 10. Skillbook Persistence — Save & Reload



 Save the learned skillbook to disk and reload it in a future session.

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    path = Path(tmpdir) / "learned_skillbook.json"

    # Save
    skillbook.save_to_file(str(path))
    print(f"Saved to {path.name}  ({path.stat().st_size} bytes)")

    # Reload
    reloaded = Skillbook.from_file(str(path))
    print(f"Reloaded: {reloaded.stats()}")
    print(f"Stats match: {reloaded.stats() == skillbook.stats()}")


 ---

 ## 11. TraceAnalyser — Learning from Pre-Recorded Traces



 `TraceAnalyser` runs the learning tail only — no Agent, no Evaluate.

 Feed it raw trace dicts (the same shape ReflectStep expects) and it

 builds a skillbook from historical data.

In [ ]:
# Simulate some pre-recorded traces (e.g., from browser-use history logs)
traces = [
    {
        "question": "Book a flight from NYC to London",
        "reasoning": "Step 1: Opened booking site. Step 2: Searched flights. Step 3: Selected cheapest option.",
        "answer": "Booked flight AA100 for $450",
        "skill_ids": [],
        "feedback": "Task succeeded in 3 steps",
        "ground_truth": None,
    },
    {
        "question": "Find the cheapest hotel in Paris",
        "reasoning": "Step 1: Opened hotel site. Step 2: Set filters. Step 3: Sorted by price. Step 4: Cookie popup blocked view.",
        "answer": "Failed: could not dismiss cookie popup",
        "skill_ids": [],
        "feedback": "Task failed — cookie popup blocked interaction after step 3",
        "ground_truth": None,
    },
    {
        "question": "Check weather in Tokyo",
        "reasoning": "Step 1: Navigated to weather.com. Step 2: Searched Tokyo. Step 3: Read forecast.",
        "answer": "Tokyo: 22C, partly cloudy",
        "skill_ids": [],
        "feedback": "Task succeeded in 3 steps — fast and accurate",
        "ground_truth": None,
    },
]

skillbook9 = Skillbook()

analyser = TraceAnalyser.from_roles(
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    skillbook=skillbook9,
)

results9 = analyser.run(traces, epochs=1)

print(f"Analysed {len(results9)} traces")
print(f"Skills learned: {skillbook9.stats()}")
for skill in skillbook9.skills()[:5]:
    print(f"  - [{skill.section}] {skill.content}")


 ### Multi-epoch trace analysis



 Each epoch re-processes all traces with the evolving skillbook.

 Early epochs extract obvious patterns; later epochs refine.

In [ ]:
skillbook10 = Skillbook()

analyser2 = TraceAnalyser.from_roles(
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    skillbook=skillbook10,
)

results10 = analyser2.run(traces, epochs=2)

print(f"Total results across 2 epochs: {len(results10)}")
print(f"Skills after 2 epochs: {skillbook10.stats()}")


 ---

 ## 12. Mixed Workflow — TraceAnalyser then ACE



 A common pattern: build an initial skillbook from historical traces,

 then deploy with live learning.

In [ ]:
# Phase 1: Build skillbook from historical data
shared_skillbook = Skillbook()

analyser_phase1 = TraceAnalyser.from_roles(
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    skillbook=shared_skillbook,
)
analyser_phase1.run(traces, epochs=1)

print(f"Phase 1 — TraceAnalyser:")
print(f"  Skills from traces: {shared_skillbook.stats()}")

# Phase 2: Deploy with live ACE learning (reuse the evolved skillbook)
ace_phase2 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=SimpleEnvironment(),
    skillbook=shared_skillbook,
)

results_phase2 = ace_phase2.run(samples[:3], epochs=1)

print(f"\nPhase 2 — ACE live learning:")
print(f"  Processed {len(results_phase2)} samples")
print(f"  Skills after live learning: {shared_skillbook.stats()}")


 ---

 ## 13. Error Handling



 Failed samples are captured in `SampleResult.error` — the pipeline

 never drops a sample silently. Other samples continue processing.

In [ ]:
bad_samples = [
    samples[0],
    Sample(question="", ground_truth=""),  # edge case: empty question
    samples[1],
]

skillbook11 = Skillbook()
ace11 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=SimpleEnvironment(),
    skillbook=skillbook11,
)

results11 = ace11.run(bad_samples, epochs=1)

for i, r in enumerate(results11, 1):
    status = "OK" if r.error is None else f"FAIL ({r.failed_at})"
    if r.output and r.output.agent_output:
        answer = r.output.agent_output.final_answer
    else:
        answer = "N/A"
    print(f"  [{i}] {status:20s}  answer={answer}")


 ---

 ## 14. Inspecting the SkillbookView



 Steps receive a read-only `SkillbookView` on the context.

 This prevents accidental mutations from within pipeline steps.

In [ ]:
sb = Skillbook()
view = SkillbookView(sb)

print(f"SkillbookView: {view}")
print(f"  len:    {len(view)}")
print(f"  stats:  {view.stats()}")
print(f"  prompt: {view.as_prompt()[:200]}...")

# Iterate over skills in the view
for skill in view:
    print(f"  - {skill.id}: {skill.content}")


 ---

 ## Summary



 | What | How |

 |------|-----|

 | Full pipeline | `ACE.from_roles(agent=..., reflector=..., skill_manager=...)` |

 | With environment | `ACE.from_roles(..., environment=SimpleEnvironment())` |

 | Without environment | `ACE.from_roles(...)` — EvaluateStep is a no-op |

 | Multi-epoch | `ace.run(samples, epochs=3)` |

 | Checkpointing | `ACE.from_roles(..., checkpoint_dir="./ckpts", checkpoint_interval=10)` |

 | Deduplication | `ACE.from_roles(..., dedup_manager=dedup, dedup_interval=5)` |

 | Trace analysis | `TraceAnalyser.from_roles(reflector=..., skill_manager=...)` |

 | Save skillbook | `ace.save("path.json")` or `skillbook.save_to_file("path.json")` |

 | Load skillbook | `Skillbook.from_file("path.json")` |

 | Manual steps | `Pipeline([AgentStep(a), EvaluateStep(e), *learning_tail(r, sm, sb)])` |

 | Learning tail | `learning_tail(reflector, skill_manager, skillbook)` |



 **Pipeline:**

 ```

 ACE:            Agent → Evaluate → Reflect → Tag → Update → Apply → [Dedup] → [Checkpoint]

 TraceAnalyser:                     Reflect → Tag → Update → Apply → [Dedup] → [Checkpoint]

 ```